In [3]:
import pandas as pd
import numpy as np

In [4]:
dataset_df = pd.read_csv("data/cvd_v2.csv", sep=',')

In [5]:
from sklearn.model_selection import train_test_split

X = dataset_df.iloc[:, :-1]
y = dataset_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 深度学习、用rnn模型，数据需要3通道，在图片中表示RGB，这里表示1
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


In [7]:
import tensorflow as tf  
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import SimpleRNN, Dense

# 创建模型
'''
该问题本质是二分类问题，故最后一层全连接层用激活函数为：sigmoid
模型结构：
    RNN：隐藏层200，激活函数：relu
    Dense：--> 100(relu) -> 1(sigmoid)
'''
# 创建模型
model = Sequential()
model.add(SimpleRNN(units=200, input_shape=(11, 1), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()


2025-05-23 22:17:26.950812: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 22:17:27.129215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748009847.208493      25 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748009847.234925      25 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748009847.385334      25 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 200)            │        40,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,601 (236.72 KB)

 Trainable params: 60,601 (236.72 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(
    optimizer=opt,
    loss='binary_crossentropy',  # 二分类问题
    metrics=['accuracy']
)


In [9]:
epochs = 100

history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)


Epoch 1/100


I0000 00:00:1748009853.996526     360 service.cc:152] XLA service 0x7f55c0004090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748009853.996566     360 service.cc:160]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2025-05-23 22:17:34.056678: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748009854.256313     360 cuda_dnn.cc:529] Loaded cuDNN version 91001
I0000 00:00:1748009854.810586     360 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1932/1932 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.6953 - loss: 0.5882 - val_accuracy: 0.7228 - val_loss: 0.5602
Epoch 2/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7332 - loss: 0.5451 - val_accuracy: 0.7259 - val_loss: 0.5584
Epoch 3/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7315 - loss: 0.5455 - val_accuracy: 0.7297 - val_loss: 0.5541
Epoch 4/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7355 - loss: 0.5419 - val_accuracy: 0.7237 - val_loss: 0.5551
Epoch 5/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7360 - loss: 0.5418 - val_accuracy: 0.7288 - val_loss: 0.5525
Epoch 6/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7320 - loss: 0.5433 - val_accuracy: 0.7291 - val_loss: 0.5534
Epoch 7/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7342 - loss: 0.5425 - val_accuracy: 0.7253 - val_loss: 0.5590
Epoch 8/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7366 - loss: 0.537

KeyboardInterrupt: 

In [ ]:
# 评估：返回的是自己在model.compile中设置，这里为accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print("socre[loss, accuracy]: ", score) # 返回为两个，一个是loss，一个是accuracy


socre[loss, accuracy]:  [0.684647798538208, 0.7118940353393555]
